In [ ]:
#import Sqllite3 and relevant libraries
import sqlite3
import csv
import pandas as pd

conn = sqlite3.connect('bike.db')
c = conn.cursor()

In [ ]:
#The dataset has been cleaned and organized. For each month in 2016 we have a csv with columns in the same ordering.
#This function creates a skeleton database to be populated with the cleaned dataset.
def create_database(conn):
    """
    Using the database connection conn, create tables appropriate for 
    storing and using Bike data.  Drops any pre-exsiting tables that would be re-defined by this procedure.
    """
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS station''')
    c.execute('''DROP TABLE IF EXISTS neighborhood''')
    c.execute('''DROP TABLE IF EXISTS weather''')
    c.execute('''DROP TABLE IF EXISTS fact_table''')
    # Create database with tables.

    c.execute('''
           CREATE TABLE station(
                station_id int,
                station_name text,
                station_latitude float,
                station_longitude float,
                station_neighborhood text,
                PRIMARY KEY(station_id), 
                FOREIGN KEY(station_neighborhood) REFERENCES neighborhood(neighborhood))''')
                
    c.execute('''
           CREATE TABLE neighborhood(
                neighborhood text,
                median_income int,
                borough text,
                PRIMARY KEY(neighborhood)) ''')
    
    c.execute('''
            CREATE TABLE weather(
                date text,
                time int,
                day int,
                humidity int,
                tempF int,
                weatherDesc text,
                windSpeedMiles int, 
                PRIMARY KEY(date, time) )''')
    
#Took out Date, Time, FOREIGN KEY(Date, Time) REFERENCES weather(Date, Time),

    c.execute('''
            CREATE TABLE fact_table(
                bike_id int,
                start_time,
                stop_time,
                start_station_id int,
                start_station_neighborhood text,
                end_station_id int,    
                end_station_neighborhood text,
                usertype text, 
                birth_year int, 
                gender int,
                trip_duration int,
                date,
                time int,
                PRIMARY KEY(bike_id, start_time, date),
                FOREIGN KEY(date, time) REFERENCES weather(date, time),
                FOREIGN KEY(start_station_id) REFERENCES station(station_id), 
                FOREIGN KEY(end_station_id) REFERENCES station(station_id),
                FOREIGN KEY(start_station_neighborhood) REFERENCES neighborhood(neighborhood),
                FOREIGN KEY(end_station_neighborhood) REFERENCES neighborhood(neighborhood))''') 
    # Commit all changes
    conn.commit()
    
# Call the create_database() function
create_database(conn)
print('done')

In [ ]:
#This funciton populates the database with the cleaned bike csv's
def populate_database(conn, csv_file):    
    reader = csv.reader(open(csv_file, encoding='latin1'))
    
    # skip header    
    next(reader, None)
    # Convert row to lower case
    for row in reader:
        row = [item.lower() for item in row]
        
        #insert into station dimension tables
        try:
            c.execute('''
                INSERT INTO station (station_id,
                station_name,
                station_latitude,
                station_longitude,
                station_neighborhood) VALUES (?,?,?,?,?)
                ''', (row[1], row[2],row[3],row[4],row[23]))
        except sqlite3.Error as e:
            conn.commit()
        
        try:
            c.execute('''
                INSERT INTO station (station_id,
                station_name,
                station_latitude,
                station_longitude,
                station_neighborhood) VALUES (?,?,?,?,?)
                ''', (row[5], row[6],row[7],row[8],row[26]))
        except sqlite3.Error as e:
            conn.commit()
        
        try:
            c.execute('''
                INSERT INTO neighborhood (neighborhood,
                median_income,
                borough) VALUES (?,?,?)
                ''', (row[23], row[25],row[24]))
        except sqlite3.Error as e:
            conn.commit()
      
        try:
            c.execute('''
                INSERT INTO neighborhood (neighborhood,
                median_income,
                borough) VALUES (?,?,?)
                ''', (row[26], row[28],row[27]))
        except sqlite3.Error as e:
            conn.commit()    
        try:
            c.execute(''' 
                INSERT INTO weather (
                date,
                time,
                day,
                humidity,
                tempF,
                weatherDesc,
                windSpeedMiles) VALUES (?,?,?,?,?,?,?)
            ''', (row[14],row[15],row[20],row[16],row[17],row[18],row[19]))
        except sqlite3.Error as e:
            conn.commit()

        try:
       
            c.execute('''
                INSERT INTO fact_table (bike_id,
                date,
                time,
                start_time,
                stop_time,
                trip_duration,
                start_station_id,
                start_station_neighborhood,
                end_station_id,    
                end_station_neighborhood,
                usertype, 
                birth_year, 
                gender) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
                      (row[29],row[14],row[16], row[12], row[13], row[0], row[1], row[23], row[5], row[26], row[9],row[10], row[11],
                       ))
        except sqlite3.Error as e:
#             print("fact_table, insert error:", e.args[0])

#     Commit to the connection in order to save the changes
            conn.commit()    
        
        


In [ ]:
#For each month we populate the database
populate_database(conn,'Income and Bike Id/april_med_bike_id.csv')

In [ ]:
# Wanted to get a sense for roughly how long it would take to populate the DB
import timeit
start = timeit.default_timer()
populate_database(conn,'Income and Bike Id/august_med_bike_id.csv')
stop = timeit.default_timer()
stop - start

In [ ]:
start = timeit.default_timer()
populate_database(conn,'Income and Bike Id/december_med_bike_id.csv')
stop = timeit.default_timer()
stop - start

In [ ]:
populate_database(conn,'Income and Bike Id/february_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/january_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/july_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/june_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/march_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/may_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/november_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/october_med_bike_id.csv')

In [ ]:
populate_database(conn,'Income and Bike Id/september_med_bike_id.csv')

In [ ]:
#Some very basic queries to see that our DB was working.
#Unfortunately queries from the factable were too much for the SQLlite to handle
%reload_ext sql
%sql sqlite:///bike.db

In [ ]:
%%sql
select * from fact_table 
where date = '2016-04-01'

In [ ]:
%%sql
SELECT * FROM weather
where humidity = 92

In [ ]:
%%sql
SELECT * FROM neighborhood

In [ ]:
%%sql
select * from station
where station_id = 254